In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from string import ascii_letters
import numpy as np
import pandas as pd
import seaborn as sns
#!pip install imbalanced-learn
#!pip3 install -U imbalanced-learn
from imblearn.over_sampling import SMOTE
import imblearn


In [2]:
loan_df = pd.read_csv("status_clean.csv")

In [3]:
loan_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106998 entries, 0 to 106997
Data columns (total 86 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Unnamed: 0                      106998 non-null  int64  
 1   loan_amnt                       106998 non-null  int64  
 2   term                            106998 non-null  object 
 3   int_rate                        106998 non-null  float64
 4   installment                     106998 non-null  float64
 5   emp_length                      100502 non-null  object 
 6   home_ownership                  106998 non-null  object 
 7   annual_inc                      106997 non-null  float64
 8   verification_status             106998 non-null  object 
 9   loan_status                     106998 non-null  object 
 10  pymnt_plan                      106998 non-null  object 
 11  title                           105907 non-null  object 
 12  zip_code        

In [4]:
'''
fico_range_low
fico_range_high
tax_liens 
pub_rec_bankruptcies
loan_status 
last_pymnt_amnt
'''

'\nfico_range_low\nfico_range_high\ntax_liens \npub_rec_bankruptcies\nloan_status \nlast_pymnt_amnt\n'

In [5]:


# loan_df1 = loan_df[["fico_range_low",
# "fico_range_high",
# "tax_liens", 
# "pub_rec_bankruptcies",
# "loan_status", 
# "last_pymnt_amnt",
# "tot_hi_cred_lim",
# "collections_12_mths_ex_med",
# "pymnt_plan",
# "avg_cur_bal",
# "dti",
# "loan_amnt",
# "int_rate",
#  "num_sats",
# "delinq_2yrs",
# "installment",
# "verification_status"]]





In [6]:
loan_df["loan_status"].value_counts()

Fully Paid                                             52842
Current                                                39517
Charged Off                                            13025
Late (31-120 days)                                       923
In Grace Period                                          369
Late (16-30 days)                                        182
Does not meet the credit policy. Status:Fully Paid        97
Does not meet the credit policy. Status:Charged Off       42
Default                                                    1
Name: loan_status, dtype: int64

In [7]:
loan_df1 = loan_df.loc[(loan_df["loan_status"] != "Does not meet the credit policy. Status:Fully Paid") & (loan_df["loan_status"] != "Does not meet the credit policy. Status:Charged Off")]




In [8]:
loan_df1["loan_status"].value_counts()

Fully Paid            52842
Current               39517
Charged Off           13025
Late (31-120 days)      923
In Grace Period         369
Late (16-30 days)       182
Default                   1
Name: loan_status, dtype: int64

In [9]:
loan_df2 = loan_df1.apply(lambda x: x.mask(x.map(x.value_counts())<= 13025, 'Late') if x.name=='loan_status' else x)

loan_df3 = loan_df2.apply(lambda x: x.mask(x.map(x.value_counts())>= 39517, 'Fully Paid') if x.name=='loan_status' else x)


In [10]:
loan_df3["loan_status"].value_counts()

Fully Paid    92359
Late          14500
Name: loan_status, dtype: int64

In [21]:
loan_df4 = loan_df3.drop("Unnamed: 0", axis=1)

loan_df4


,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,6500,36 months,9.75,208.98,3 years,RENT,80000.0,Not Verified,Fully Paid,n,...,0.0,1.0,95.8,100.0,1.0,0.0,32332.0,14444.0,3800.0,16732.0
1,4000,36 months,6.83,123.20,6 years,RENT,30000.0,Not Verified,Fully Paid,n,...,0.0,1.0,88.5,0.0,0.0,0.0,74521.0,79942.0,3700.0,66821.0
2,20000,36 months,9.17,637.58,10+ years,RENT,80000.0,Source Verified,Fully Paid,n,...,0.0,1.0,100.0,100.0,0.0,0.0,18000.0,15208.0,16000.0,0.0
3,15000,60 months,15.61,361.67,10+ years,RENT,45000.0,Verified,Fully Paid,n,...,0.0,0.0,100.0,100.0,1.0,3.0,42638.0,23022.0,14400.0,27029.0
4,24000,60 months,15.59,578.42,9 years,MORTGAGE,64000.0,Source Verified,Fully Paid,n,...,0.0,3.0,95.1,75.0,0.0,0.0,208862.0,127502.0,15400.0,109688.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106993,10000,36 months,6.67,307.27,< 1 year,OWN,34655.0,Source Verified,Fully Paid,n,...,0.0,1.0,100.0,100.0,0.0,0.0,16100.0,10380.0,6000.0,0.0
106994,13000,36 months,12.13,432.60,< 1 year,MORTGAGE,150000.0,Source Verified,Fully Paid,n,...,0.0,1.0,87.0,37.5,0.0,0.0,113278.0,96342.0,10600.0,81978.0
106995,7000,36 months,8.46,220.85,10+ years,MORTGAGE,75000.0,Source Verified,Fully Paid,n,...,0.0,1.0,85.0,0.0,0.0,0.0,35883.0,20459.0,9650.0,16793.0
106996,12000,60 months,9.49,251.97,10+ years,MORTGAGE,66386.0,Not Verified,Fully Paid,n,...,0.0,2.0,100.0,20.0,0.0,0.0,214053.0,21015.0,31400.0,15553.0


In [22]:
dummies_df  = pd.get_dummies(loan_df4)

In [23]:
dummies_df

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,...,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY,initial_list_status_f,initial_list_status_w
0,6500,9.75,208.98,80000.0,9.17,0.0,675,679,0.0,72.0,...,0,0,0,0,0,0,0,0,0,1
1,4000,6.83,123.20,30000.0,15.68,0.0,695,699,1.0,NaN,...,0,0,0,0,0,0,0,0,0,1
2,20000,9.17,637.58,80000.0,6.41,0.0,690,694,1.0,NaN,...,0,0,0,0,0,0,0,0,1,0
3,15000,15.61,361.67,45000.0,23.57,0.0,660,664,0.0,NaN,...,0,0,0,0,0,0,0,0,0,1
4,24000,15.59,578.42,64000.0,25.86,0.0,665,669,1.0,69.0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106993,10000,6.67,307.27,34655.0,8.17,0.0,680,684,1.0,NaN,...,0,0,0,0,0,0,0,0,1,0
106994,13000,12.13,432.60,150000.0,11.74,0.0,680,684,0.0,27.0,...,0,0,0,0,0,0,0,0,0,1
106995,7000,8.46,220.85,75000.0,9.81,2.0,690,694,1.0,14.0,...,0,0,0,0,0,0,0,0,0,1
106996,12000,9.49,251.97,66386.0,13.56,0.0,750,754,0.0,NaN,...,0,0,0,0,0,0,0,0,0,1


In [24]:
from sklearn.model_selection import train_test_split
# X = train_df
X = dummies_df.drop(["loan_status_Fully Paid","loan_status_Late"], axis=1)

y = dummies_df["loan_status_Fully Paid"]

oversample = SMOTE()

X, y = oversample.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier

from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

# clf = AdaBoostClassifier(n_estimators=100)

# reg = Lasso(max_iter=1000).fit(X_train_scaled, y_train)
classifier.fit(X_train_scaled, y_train)
#random forest
#clf.fit(X_train_scaled, y_train)

clf = RandomForestClassifier(random_state=42, n_estimators=50).fit(X_train_scaled, y_train)

#forest/adaboost
print(f'Training Score: {clf.score(X_train_scaled, y_train)}')
print(f'Testing Score: {clf.score(X_test_scaled, y_test)}')

#logistic regression
# print(f'Training Score: {classifier.score(X_train_scaled, y_train)}')
# print(f'Testing Score: {classifier.score(X_test_scaled, y_test)}')

ValueError: Input X contains NaN.
SMOTE does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
y_test.value_counts()

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


y_true = y_test

y_pred = classifier.predict(X_test)

confusion_matrix(y_true, y_pred)

confusion_matrix(y_test, y_pred)